*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will create a book recommendation algorithm using **K-Nearest Neighbors**.

You will use the [Book-Crossings dataset](http://www2.informatik.uni-freiburg.de/~cziegler/BX/). This dataset contains 1.1 million ratings (scale of 1-10) of 270,000 books by 90,000 users. 

After importing and cleaning the data, use `NearestNeighbors` from `sklearn.neighbors` to develop a model that shows books that are similar to a given book. The Nearest Neighbors algorithm measures distance to determine the “closeness” of instances.

Create a function named `get_recommends` that takes a book title (from the dataset) as an argument and returns a list of 5 similar books with their distances from the book argument.

This code:

`get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")`

should return:

```
[
  'The Queen of the Damned (Vampire Chronicles (Paperback))',
  [
    ['Catch 22', 0.793983519077301], 
    ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448656558990479], 
    ['Interview with the Vampire', 0.7345068454742432],
    ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5376338362693787],
    ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178412199020386]
  ]
]
```

Notice that the data returned from `get_recommends()` is a list. The first element in the list is the book title passed in to the function. The second element in the list is a list of five more lists. Each of the five lists contains a recommended book and the distance from the recommended book to the book passed in to the function.

If you graph the dataset (optional), you will notice that most books are not rated frequently. To ensure statistical significance, remove from the dataset users with less than 200 ratings and books with less than 100 ratings.

The first three cells import libraries you may need and the data to use. The final cell is for testing. Write all your code in between those cells.

In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2022-08-25 15:57:44--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.6’

book-crossings.zip. 100%[===================>]  24.88M   115MB/s    in 0.2s    

2022-08-25 15:57:45 (115 MB/s) - ‘book-crossings.zip.6’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: BX-Book-Ratings.csv     
replace BX-Books.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: BX-Books.csv            y

replace BX-Users.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: BX-Users.csv            


In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
user_ratingCount = df_ratings.groupby(by = ['user']).count().reset_index().drop(['isbn'], axis = 1).rename(columns = {'rating': 'user_count'})[['user','user_count']]
user_ratingCount

,user,user_count
0,2,1
1,7,1
2,8,18
3,9,3
4,10,2
...,...,...
105278,278846,2
105279,278849,4
105280,278851,23
105281,278852,1


In [ ]:
book_ratingCount = df_ratings.groupby(by = ['isbn'])['rating'].count().reset_index().rename(columns = {'rating': 'rating_count'})[['isbn', 'rating_count']]
                    
book_ratingCount

,isbn,rating_count
0,0330299891,2
1,0375404120,2
2,0586045007,1
3,9022906116,2
4,9032803328,1
...,...,...
340551,cn113107,1
340552,ooo7156103,1
340553,§423350229,1
340554,´3499128624,1


In [ ]:
df0 = df_ratings.merge(user_ratingCount, left_on= 'user', right_on='user', how='left')
df0 = df0.merge(book_ratingCount, left_on= 'isbn', right_on='isbn', how='left')
df0 = df0[df0.user_count >= 200][df0.rating_count >= 100]
df0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


,user,isbn,rating,user_count,rating_count
1456,277427,002542730X,10.0,497,171
1469,277427,0060930535,0.0,497,494
1471,277427,0060934417,0.0,497,350
1474,277427,0061009059,9.0,497,291
1484,277427,0140067477,0.0,497,189
...,...,...,...,...,...
1147304,275970,0804111359,0.0,1376,167
1147436,275970,140003065X,0.0,1376,157
1147439,275970,1400031346,0.0,1376,106
1147440,275970,1400031354,0.0,1376,202


In [ ]:
# add your code here - consider creating a new cell for each section of cod
df = pd.merge(df_books, df0, on='isbn')
df.drop('author', axis = 1, inplace = True)
df

,isbn,title,user,rating,user_count,rating_count
0,0440234743,The Testament,277478,0.0,214,422
1,0440234743,The Testament,2977,0.0,232,422
2,0440234743,The Testament,3363,0.0,901,422
3,0440234743,The Testament,7346,9.0,1007,422
4,0440234743,The Testament,9856,0.0,201,422
...,...,...,...,...,...,...
49512,0515135739,Eleventh Hour: An FBI Thriller (FBI Thriller (...,236283,0.0,1329,102
49513,0515135739,Eleventh Hour: An FBI Thriller (FBI Thriller (...,251613,0.0,1124,102
49514,0515135739,Eleventh Hour: An FBI Thriller (FBI Thriller (...,252071,0.0,424,102
49515,0515135739,Eleventh Hour: An FBI Thriller (FBI Thriller (...,256407,0.0,619,102


In [ ]:
df = df.dropna(axis = 0, subset = ['title'])


In [ ]:
df.describe()

,user,rating,user_count,rating_count
count,49517.000000,49517.000000,49517.000000,49517.000000
mean,141006.087768,2.019084,1092.612820,251.883373
std,80902.411522,3.572597,1770.574691,249.802607
min,254.000000,0.000000,200.000000,100.000000
25%,70415.000000,0.000000,319.000000,133.000000
50%,140358.000000,0.000000,552.000000,181.000000
75%,213350.000000,3.000000,1172.000000,284.000000
max,278418.000000,10.000000,13602.000000,2502.000000


In [ ]:
df1 = df.drop_duplicates(['title', 'user'])
df1

,isbn,title,user,rating,user_count,rating_count
0,0440234743,The Testament,277478,0.0,214,422
1,0440234743,The Testament,2977,0.0,232,422
2,0440234743,The Testament,3363,0.0,901,422
3,0440234743,The Testament,7346,9.0,1007,422
4,0440234743,The Testament,9856,0.0,201,422
...,...,...,...,...,...,...
49512,0515135739,Eleventh Hour: An FBI Thriller (FBI Thriller (...,236283,0.0,1329,102
49513,0515135739,Eleventh Hour: An FBI Thriller (FBI Thriller (...,251613,0.0,1124,102
49514,0515135739,Eleventh Hour: An FBI Thriller (FBI Thriller (...,252071,0.0,424,102
49515,0515135739,Eleventh Hour: An FBI Thriller (FBI Thriller (...,256407,0.0,619,102


In [ ]:
df1_pivot = df1.pivot(index = 'title', columns = 'user', values = 'rating').fillna(0)
df1_pivot

user,254,2276,2766,2977,3363,4017,4385,6242,6251,6323,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Beautiful Mind: The Life of Mathematical Genius and Nobel Laureate John Nash,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Without Remorse,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,7.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df1_matrix = csr_matrix(df1_pivot)
df1_matrix

<673x888 sparse matrix of type '<class 'numpy.float32'>'
	with 12423 stored elements in Compressed Sparse Row format>

In [ ]:
nbrs = NearestNeighbors(n_neighbors= 6, metric = 'cosine', algorithm = 'brute')
nbrs.fit(df1_matrix)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=6)

In [ ]:
dist, ind =nbrs.kneighbors(df1_pivot.loc["1984"].values.reshape(1, -1))
df1_pivot.index[ind[0,1]]

"The Handmaid's Tale"

In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  distances, indices = nbrs.kneighbors(df1_pivot.loc[book].values.reshape(1, -1))
  recommended_books = [book,[[df1_pivot.index[indices[0,i]], distances[0,i]] for i in reversed(range(1, indices.shape[1]))]]


  return recommended_books

In [ ]:
get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")

['The Queen of the Damned (Vampire Chronicles (Paperback))',
 [['Catch 22', 0.7939835],
  ['The Witching Hour (Lives of the Mayfair Witches)', 0.74486566],
  ['Interview with the Vampire', 0.73450685],
  ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.53763384],
  ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.51784116]]]

Use the cell below to test your function. The `test_book_recommendation()` function will inform you if you passed the challenge or need to keep trying.

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016211], ['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7234864]]]
You passed the challenge! 🎉🎉🎉🎉🎉
